In [21]:

import mqttools, threading, asyncio, json
import numpy as np

class ComplexJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.ndarray, np.number)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj) 

async def publishTopic(host, port, topic, message):

    client = mqttools.Client(host, port, connect_delays=[], response_timeout=2)            
    rc = client.client_id
    try:
        await client.start()
        client.publish(mqttools.Message(topic, message))            
    except Exception as ex:
        rc = -1             
    finally:
        await client.stop() 
                            
    return rc   

host, port = 'localhost', 10098
topic = '/hot/ecat/value'


In [142]:

data = {"source": "el7201Worker", "target": 1, "value": { "control": "00000110" } }

#data = {"source": "el7201Worker", "target": 1, "value": { "velocity": -5_000_000 } }

loop = asyncio.new_event_loop()
threading.Thread(target=loop.run_forever).start()
message = json.dumps(data, cls=ComplexJSONEncoder).encode('ascii')
asyncio.run_coroutine_threadsafe(publishTopic(host, port, topic, message), loop)


<Future at 0x299d45a4dd0 state=pending>